In [1]:
\cd ../qenv/

In [37]:
\l test.q

"###################################################################################################"
"                                             TEST"
"###################################################################################################"
"==================================================================================================="
"37) .engine.logic.inventory.Fill (UNIT) :READY"
`Function for deriving the exec cost from the qty and the price
"---------------------------------------------------------------------------------------------------"
caseId state dscr                                            
-------------------------------------------------------------
256    ERROR INVERSE:flat to long: UPL: 0, RPL:0 ONE POSITION
"                                                                                                                                                                                                                                                

In [13]:
iv:first flip .bam.x

In [27]:
.engine.model.inventory.Inventory

aId side| ordQty ordVal ordLoss amt iw mm posVal rpnl avgPrice execCost totEnt upnl lev
--------| -----------------------------------------------------------------------------
0   -1  | 0             0       10     0                                10     0       
0   1   | 0             0       10     0                                10     0       


In [17]:
.engine.logic.contract.UnrealizedPnl    :{[contractType;markPrice;faceValue;multiplier;amt;isignum;avgPrice] 
      7h$($[contractType=0;.engine.logic.contract.inverse.UnrealizedPnl[amt;isignum;avgPrice;markPrice;faceValue;multiplier]; 
            contractType=0;.engine.logic.contract.inverse.UnrealizedPnl[amt;isignum;avgPrice;markPrice;faceValue;multiplier]; 
          contractType=0;.engine.logic.contract.inverse.UnrealizedPnl[amt;isignum;avgPrice;markPrice;faceValue;multiplier]; 
      'nyi] | 0)
    };

In [16]:
.engine.logic.contract.RealizedPnl    :{[contractType;fillQty;fillPrice;isignum;avgPrice;faceValue;multiplier]
    7h$($[contractType=0;.engine.logic.contract.inverse.RealizedPnl[fillQty;fillPrice;isignum;avgPrice;faceValue;multiplier];
      contractType=1;.engine.logic.contract.linear.RealizedPnl[fillQty;fillPrice;isignum;avgPrice;faceValue;multiplier];
      contractType=2;.engine.logic.contract.quanto.RealizedPnl[fillQty;fillPrice;isignum;avgPrice;faceValue;multiplier];
      'nyi] | 0)
    };

In [47]:
select ivId.ordVal from .bam.x

ordVal|  


In [ ]:
.engine.logic.contract.ExecCost    :{[contractType;price;qty;multiplier]

fId     | 0          1         
price   | 1000       1000      
qty     | 100        100       
reduce  | 0          1         
ismaker | 0          1         
side    | 1          -1        
iId     | 0          0         
oId     | 0          1         
ivId    | 1          0         
aId     | 0          0         
avgPrice| 0          0         
execCost| 10000000   10000000  
ordLoss | 0          0         
ordVal  | -100000000 -100000000
ordQty  | -100       -100      
amt     | 110        110       
totEnt  | 210        210       


In [39]:
iviv:![![.bam.x;();0b;`avgPrice`execCost`ordLoss`ordVal`ordQty`amt`totEnt!(
        ((';`.engine.logic.contract.AvgPrice);`iId.cntTyp;`side;`ivId.execCost;`ivId.totEnt;`iId.smul); // TODO dependent
        ((';`.engine.logic.contract.ExecCost);`iId.cntTyp;`price;`qty;`iId.smul);
        (-;`ivId.ordLoss;(`.engine.logic.contract.Loss;`iId.mkprice;`qty;((+/);`qty;`price)));
        (-;`ivId.ordVal;((*/);`qty;`price));
        (-;`ivId.ordQty;`qty);
        (+;`ivId.amt;`qty);
        ((+/);`ivId.totEnt;`qty)
    )];
    ();0b;`upnl`rpnl!(
    ((';`.engine.logic.contract.UnrealizedPnl);`iId.cntTyp;`iId.mkprice;`iId.faceValue;`iId.smul;`ivId.amt;`side;`ivId.avgPrice);
    ((';`.engine.logic.contract.RealizedPnl);`iId.cntTyp;`qty;`price;`side;`ivId.avgPrice;`iId.faceValue;`iId.smul))]

In [59]:
.q.cl:{x!y[x]}

In [67]:
.engine.model.inventory.Inventory upsert flip 

aId side| ordQty ordVal     ordLoss amt iw mm posVal rpnl avgPrice execCost totEnt upnl lev
--------| ---------------------------------------------------------------------------------
0   -1  | -100   -100000000 0       110    0         0    0        10000000 210    0       
0   1   | -100   -100000000 0       110    0         0    0        10000000 210    0       


In [68]:
.bam.x

fId    | 0    1   
price  | 1000 1000
qty    | 100  100 
reduce | 0    1   
ismaker| 0    1   
side   | 1    -1  
iId    | 0    0   
oId    | 0    1   
ivId   | 1    0   
aId    | 0    0   


In [ ]:
.engine.logic.inventory.Fill:{ // TODO simple select

    iv:?[?[x;();0b;(
                (`.engine.logic.contract.UnrealizedPnl;`cntTyp;`mkprice;`faceValue;`smul;`amt;`side;`avgPrice);
                (`.engine.logic.contract.RealizedPnl;`iId.cntTyp;`qty;`price;`side;`ivId.avgPrice;`iId.faceValue;`iId.smul)
        )];();0b;(
        (`.engine.logic.contract.AvgPrice);
        (`.engine.logic.contract.ExecCost);
        (-;`ivId.ordLoss;(`.engine.logic.contract.Loss;));
        (-;`ivId.ordVal;(`.engine.logic.contract.Value;));
        (-;`ivId.ordQty;qty);
        (+;`amt;`dlt);
        (sum;`ivId.totalEntry;(max;`dlt;0))
        )];

    .engine.model.inventory.Update iv;
    .engine.EmitA[`inventory;];

    		/if[abs[iv[`amt]]=0;iv[`avgPrice`execCost`totalEntry]:0];

    };



In [40]:
e:flip(iv`kind;iv`time;flip iv`aId`side`time`amt`rpnl`avgPrice`upnl;7h$iv`aId)

In [44]:
x:.bam.x

In [46]:
x[`kind]:`fill

In [47]:
e:flip(x`kind;x`time;flip x`time`price`qty`side;7h$x`aId)

In [50]:
.engine.egress.Events

time                    kind      datum                                                                 aId
-----------------------------------------------------------------------------------------------------------
2020.12.13T14:00:29.774 inventory `.engine.model.account.Account$0 1  2020.12.13T14:00:29.774 110 0 0 0 0  
2020.12.13T14:00:29.774 inventory `.engine.model.account.Account$0 -1 2020.12.13T14:00:29.774 110 0 0 0 0  


In [58]:
e

`fill 2020.12.13T14:00:29.774 (2020.12.13T14:00:29.774;1000;100;1)  0
`fill 2020.12.13T14:00:29.774 (2020.12.13T14:00:29.774;1000;100;-1) 0


In [66]:
m:x

In [71]:
x:m

In [67]:
x:first flip x

In [73]:
x:flip x

In [80]:
x`time;x`time`price`qty`side

2020.12.13T14:00:29.774 2020.12.13T14:00:29.774
1000                    1000                   
100                     100                    
1                       -1                     


In [86]:
.engine.EmitA'[x`kind;scl[`time`price`qty`side;flip x];7h$x`aId];

In [83]:
delete from `.engine.egress.Events

`.engine.egress.Events


In [87]:
.engine.egress.Events

time kind datum aId
-------------------


In [4]:
iv[`kind]:`inventory;
show ;
.engine.EmitA[iv`kind;s`time;flip s`aId`side`time`amt`rpnl`avgPrice`upnl;iv`aId];

[0;31ms[0m: [0;31ms[0m

In [ ]:
.event.Event:{

    }

In [ ]:
.model.Account: {

    }

In [ ]:
.event.Event: {}

.event.Failure :{}
.event.Account :{}
.event.Inventory :{}
.event.Deposit :{}
.event.Withdraw :{}
.event.Funding : {}
.event.Mark:{}
.event.Settlement:{}
.event.PriceLimit :{}
.event.Level:{}
.event.Trade:{}
.event.Order:{}
.event.Instrument:{}
.event.Fill:{}
.event.Liquidation:{}